In [32]:
"""
Summaries of USFS TreeMap linked to FIA plot data
Emphasis on 
    - Metrics of forest composition
    - Ecological gradients of species dominance
    - Forest structure (basal area, QMD, TPA, etc.)

Aggregate these statistics to FRP gridcells.

Author: maxwell.cook@colorado.edu
"""

import os, sys, time
import pandas as pd
import rioxarray as rxr
import geopandas as gpd

# Custom functions
sys.path.append(os.path.join(os.getcwd(),'code/'))
from __functions import *

proj = 'EPSG:5070'

maindir = '/Users/max/Library/CloudStorage/OneDrive-Personal/mcook/'
projdir = os.path.join(maindir, 'aspen-fire/Aim2/')

print("Ready to go !")

Ready to go !


In [33]:
# Load the TreeMap (ca. 2016)
fp = os.path.join(maindir,'data/landcover/USFS/RDS_TreeMap/TreeMap2016.tif')
treemap_da = rxr.open_rasterio(fp, masked=True, cache=False, chunks='auto').squeeze()
# Grab some raster metadata
shp, gt, wkt, nd = treemap_da.shape, treemap_da.spatial_ref.GeoTransform, treemap_da.rio.crs, treemap_da.rio.nodata
print(
    f"Shape: {shp}; \n"
    f"GeoTransform: {gt}; \n"
    f"WKT: {wkt}; \n"
    f"NoData Value: {nd}; \n"
    f"Data Type: {treemap_da[0].dtype}")
gc.collect() # clean up

Shape: (97383, 154221); 
GeoTransform: -2362845.0 30.0 0.0 3180555.0 0.0 -30.0; 
WKT: EPSG:5070; 
NoData Value: nan; 
Data Type: float64


24

In [34]:
# load and prepare our study region for cropping TreeMap
fp = os.path.join(projdir,'data/spatial/raw/boundaries/na_cec_eco_l3_srme.gpkg')
srm = gpd.read_file(fp)
# Crop the raster by the SRM bounds
bounds = srm.total_bounds # total bounds of ecoregion
treemap_da_c = treemap_da.rio.clip_box(
    minx=bounds[0]+10000, # +10km buffer
    miny=bounds[1]+10000, 
    maxx=bounds[2]+10000, 
    maxy=bounds[3]+10000
)
print(f"Cropped TreeMap to SRM bounds w/ 10km buffer.")
del treemap_da, bounds
gc.collect() # clean up

Cropped TreeMap to SRM bounds w/ 10km buffer.


1094

In [37]:
# load the aggregated FRP grid
fp = os.path.join(projdir,'data/spatial/mod/VIIRS/viirs_snpp_jpss1_afd_latlon_fires_pixar_gridstats.gpkg')
grid = gpd.read_file(fp)
grid.columns

Index(['grid_index', 'grid_area', 'afd_count', 'unique_days', 'overlap',
       'frp_csum', 'frp_max', 'frp_min', 'frp_mean', 'frp_p90', 'frp_first',
       'day_max_frp', 'dt_max_frp', 'first_obs_date', 'last_obs_date',
       't4_max', 't4_mean', 't5_max', 't5_mean', 'day_count', 'night_count',
       'frp_max_day', 'frp_max_night', 'frp_csum_day', 'frp_csum_night',
       'frp_mean_day', 'frp_mean_night', 'frp_p90_day', 'frp_p90_night',
       'frp_first_day', 'frp_first_night', 'Fire_ID', 'Fire_Name', 'geometry'],
      dtype='object')

In [50]:
# get the count of unique "tm_id" from TreeMap in grids
t0 = time.time()

# see __functions.py
grid_tmid = compute_band_stats(grid, treemap_da_c, 'grid_index', attr='tm_id')

t1 = (time.time() - t0) / 60
print(f"Total elapsed time: {t1:.2f} minutes.")
print("\n~~~~~~~~~~\n")

Total elapsed time: 0.58 minutes.

~~~~~~~~~~



In [51]:
# tidy columns
grid_tmid['count'] = grid_tmid['count'].astype(int)
grid_tmid['total_pixels'] = grid_tmid['total_pixels'].astype(int)
grid_tmid.rename(columns={'total_pixels': 'forest_pixels'}, inplace=True)
grid_tmid.head()

,grid_index,tm_id,count,forest_pixels,pct_cover
0,1969858,12582,1,196,0.510204
1,1969858,12665,3,196,1.530612
2,1969858,20313,3,196,1.530612
3,1969858,45478,13,196,6.632653
4,1969858,45667,2,196,1.020408


In [52]:
# load the Tree Table
fp = os.path.join(maindir,'data/landcover/USFS/RDS_TreeMap/TreeMap2016_tree_table.csv')
tree_tbl = pd.read_csv(fp)
tree_tbl.columns

Index(['tm_id', 'CN', 'STATUSCD', 'TPA_UNADJ', 'SPCD', 'COMMON_NAME',
       'SCIENTIFIC_NAME', 'SPECIES_SYMBOL', 'DIA', 'HT', 'ACTUALHT', 'CR',
       'SUBP', 'TREE', 'AGENTCD'],
      dtype='object')

In [53]:
# join to the grid data
grid_trees = grid_tmid.merge(tree_tbl, on='tm_id', how='left')
grid_trees.head()

,grid_index,tm_id,count,forest_pixels,pct_cover,CN,STATUSCD,TPA_UNADJ,SPCD,COMMON_NAME,SCIENTIFIC_NAME,SPECIES_SYMBOL,DIA,HT,ACTUALHT,CR,SUBP,TREE,AGENTCD
0,1969858,12582,1,196,0.510204,37273076010690,2,6.018046,15,white fir,Abies concolor,ABCO,8.3,36.0,36.0,NaN,1,2,NaN
1,1969858,12582,1,196,0.510204,37273076010690,1,6.018046,15,white fir,Abies concolor,ABCO,7.1,30.0,30.0,32.0,1,3,NaN
2,1969858,12582,1,196,0.510204,37273076010690,2,6.018046,15,white fir,Abies concolor,ABCO,6.7,27.0,9.0,NaN,1,4,NaN
3,1969858,12582,1,196,0.510204,37273076010690,1,6.018046,15,white fir,Abies concolor,ABCO,10.2,33.0,33.0,40.0,1,5,NaN
4,1969858,12582,1,196,0.510204,37273076010690,1,6.018046,15,white fir,Abies concolor,ABCO,8.4,32.0,32.0,39.0,1,7,NaN


In [67]:
# identify the dominant forest species for each "tm_id"
spp_dominance = (
    grid_trees.groupby(['tm_id', 'COMMON_NAME'])['TREE']  # Sum tree counts (or use another metric)
    .sum()
    .reset_index()
    .sort_values(['tm_id', 'TREE'], ascending=[True, False])
    .drop_duplicates('tm_id')  # Keep only the top species per tm_id
    .rename(columns={'COMMON_NAME': 'MajoritySpp'})
)

# join back to the tm_id summary
grid_tmid_spp = grid_tmid.merge(spp_dominance[['tm_id', 'MajoritySpp']], on='tm_id', how='left')

# identify the landscape proportion of dominant species
spp_pr = (
    grid_tmid_spp.groupby('MajoritySpp')['count']
    .sum()
    .reset_index()
    .rename(columns={'count': 'maj_spp_count'})
    .sort_values(by='maj_spp_count', ascending=False)
)

# calculate the fraction
spp_pr['fraction'] = spp_pr['maj_spp_count'] / spp_pr['maj_spp_count'].sum()
spp_pr = spp_pr.sort_values(by='fraction', ascending=False)

# Identify species contributing 97% of the burned area
spp_pr['c_fraction'] = spp_pr['fraction'].cumsum()
top_species = spp_pr[spp_pr['c_fraction'] <= 0.99]
print(f"\nSpecies contributing to 97% of the burned area:\n{top_species}\n")


Species contributing to 97% of the burned area:
                    MajoritySpp  maj_spp_count  fraction  c_fraction
51               lodgepole pine        2201614  0.255094    0.255094
61               ponderosa pine        1968682  0.228105    0.483198
62                quaking aspen         985166  0.114148    0.597346
71                subalpine fir         704124  0.081585    0.678931
11             Engelmann spruce         543260  0.062946    0.741877
13                   Gambel oak         485799  0.056288    0.798165
9                   Douglas-fir         477115  0.055282    0.853446
36  common or two-needle pinyon         221391  0.025652    0.879098
79                    white fir         218386  0.025304    0.904402
42                    grand fir         213374  0.024723    0.929125
26                 Utah juniper         109474  0.012684    0.941809
37                 corkbark fir          95915  0.011113    0.952922
21       Rocky Mountain juniper          94477  0.0109

In [68]:
# do some species regrouping

# pinon-juniper
pj = grid_trees[grid_trees['COMMON_NAME'].str.contains('pinyon|juniper', case=False, na=False)]
print(pj['COMMON_NAME'].unique())
# replace the species names
spp_remap = {name: 'pinon-juniper' for name in pj['COMMON_NAME'].unique()}

# spruce-fir
spruce_fir = grid_trees[grid_trees['COMMON_NAME'].str.contains(' fir|spruce', case=False, na=False)]
print(spruce_fir['COMMON_NAME'].unique())
# replace the species names
spp_remap.update({name: 'spruce-fir' for name in spruce_fir['COMMON_NAME'].unique()})

# oak woodland
oak = grid_trees[grid_trees['COMMON_NAME'].str.contains('oak|mountain-mahogany', case=False, na=False)]
print(oak['COMMON_NAME'].unique())
# replace the species names
spp_remap.update({name: 'oak-scrub' for name in spruce_fir['COMMON_NAME'].unique()})

# apply the new mapping
grid_tree_r = grid_trees.copy()
grid_tree_r['ForestType'] = grid_tree_r['COMMON_NAME'].map(spp_remap).fillna(grid_tree_r['COMMON_NAME'])

['Rocky Mountain juniper' 'western juniper' 'common or two-needle pinyon'
 'Utah juniper' 'oneseed juniper' 'alligator juniper' 'singleleaf pinyon'
 'California juniper' 'Arizona pinyon pine' 'redberry juniper'
 'Pinchot juniper' 'Ashe juniper']
['white fir' 'grand fir' 'Engelmann spruce' 'subalpine fir' 'blue spruce'
 'corkbark fir' 'balsam fir' 'California red fir' 'white spruce'
 'Pacific silver fir' 'black spruce' 'noble fir' 'Shasta red fir']
['Gambel oak' 'curlleaf mountain-mahogany' 'Arizona white oak' 'gray oak'
 'California black oak' 'canyon live oak' 'bur oak' 'Oregon white oak'
 'Emory oak' 'northern pin oak' 'northern red oak' 'Mexican blue oak'
 'live oak' 'California white oak' 'blue oak']


In [70]:
# identify the dominant forest species for each "tm_id"
spp_dominance = (
    grid_tree_r.groupby(['tm_id', 'ForestType'])['TREE']  # Sum tree counts (or use another metric)
    .sum()
    .reset_index()
    .sort_values(['tm_id', 'TREE'], ascending=[True, False])
    .drop_duplicates('tm_id')  # Keep only the top species per tm_id
    .rename(columns={'ForestType': 'MajoritySpp'})
)

# join back to the tm_id summary
grid_tmid_spp = grid_tmid.merge(spp_dominance[['tm_id', 'MajoritySpp']], on='tm_id', how='left')

# identify the landscape proportion of dominant species
spp_pr = (
    grid_tmid_spp.groupby('MajoritySpp')['count']
    .sum()
    .reset_index()
    .rename(columns={'count': 'maj_spp_count'})
    .sort_values(by='maj_spp_count', ascending=False)
)

# calculate the fraction
spp_pr['fraction'] = spp_pr['maj_spp_count'] / spp_pr['maj_spp_count'].sum()
spp_pr = spp_pr.sort_values(by='fraction', ascending=False)

# Identify species contributing 97% of the burned area
spp_pr['c_fraction'] = spp_pr['fraction'].cumsum()
top_species = spp_pr[spp_pr['c_fraction'] <= 0.97]
print(f"\nSpecies contributing to 97% of the burned area:\n{top_species}\n")


Species contributing to 97% of the burned area:
       MajoritySpp  maj_spp_count  fraction  c_fraction
36  lodgepole pine        2166457  0.251020    0.251020
46  ponderosa pine        1921698  0.222661    0.473681
41       oak-scrub        1874295  0.217168    0.690849
47   quaking aspen         947308  0.109761    0.800611
44   pinon-juniper         579363  0.067129    0.867740
9       Gambel oak         473871  0.054906    0.922646



In [48]:
spp_pct_cover.head()

,MajoritySpp,maj_spp_pct,pct_csum
61,ponderosa pine,1.364142e+06,1.364142e+06
51,lodgepole pine,1.304364e+06,2.668507e+06
62,quaking aspen,6.859849e+05,3.354492e+06
71,subalpine fir,4.345706e+05,3.789062e+06
13,Gambel oak,3.419559e+05,4.131018e+06


In [36]:
# test tree table
tree_tbl[tree_tbl['tm_id'] == 21404][['tm_id','COMMON_NAME','DIA','HT','CR','TREE']]

,tm_id,COMMON_NAME,DIA,HT,CR,TREE
518539,21404,lodgepole pine,8.6,33.0,85.0,1
518540,21404,Engelmann spruce,6.0,24.0,90.0,2
518541,21404,lodgepole pine,13.4,41.0,85.0,3
518542,21404,lodgepole pine,9.1,31.0,85.0,4
518543,21404,Engelmann spruce,8.5,35.0,90.0,5
518544,21404,lodgepole pine,6.7,29.0,80.0,6
518545,21404,Engelmann spruce,6.7,35.0,90.0,7
518546,21404,lodgepole pine,9.4,47.0,85.0,1
518547,21404,Engelmann spruce,6.3,30.0,90.0,2
518548,21404,subalpine fir,8.4,39.0,90.0,3
